In [1]:
import pandas as pd
import os 
import numpy as np
import re
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
# Change directory
directory = os.chdir("/Users/joshuawong/Documents/GitHub/job-posting-analysis/data/raw/Linkedin")
files = os.listdir(".")
files

['cloud_posts_steph.csv',
 'cloud_posts_joshua.csv',
 'cloud_links.csv',
 'cloud_posts_lingjia.csv',
 'cloud_posts_gihan.csv',
 'cloud_posts_darren.csv']

In [3]:
# get all relevant files in df
# Change filename

relevant_csv = [x for x in files if 'posts' in x]
relevant_csv

['cloud_posts_steph.csv',
 'cloud_posts_joshua.csv',
 'cloud_posts_lingjia.csv',
 'cloud_posts_gihan.csv',
 'cloud_posts_darren.csv']

In [25]:
# Combine all csv
df = pd.concat(map(pd.read_csv, relevant_csv), ignore_index=True, axis=0)
df

,jobTitle,jobLocation,applicantsCount,viewsCount,jobPosterProfileUrl,postedAt,appliesClosed,applyUrl,jobDescription,remoteAllowed,...,matchedSkills,jobFunctions,jobIndustries,appliesClosedAt,jobType,jobUrl,jobImageUrl,timestamp,query,error
0,Cloud Security Architect (National Strategic P...,"Singapore, Singapore",1.0,23.0,NaN,2022-03-11T13:49:39.000Z,False,https://sggovterp.wd102.myworkdayjobs.com/en-U...,The Government Technology Agency (GovTech) aim...,False,...,NaN,Information Technology,Information Technology & Services,2022-04-10T13:49:38.000Z,Full-time,https://www.linkedin.com/jobs/view/2952531304/,https://media-exp1.licdn.com/dms/image/C4D0BAQ...,2022-03-16T00:39:42.612Z,https://www.linkedin.com/jobs/view/2952531304/,NaN
1,Senior Consultant - Cloud Solutions Architect ...,"Singapore, Singapore",126.0,988.0,NaN,2022-02-23T11:38:28.000Z,False,https://jobs.sea.deloitte.com/job/Singapore-Se...,"At Deloitte, we offer a unique and exceptional...",False,...,NaN,"Consulting, Information Technology, Sales","Management Consulting, Information Technology ...",2022-06-03T13:30:50.000Z,Full-time,https://www.linkedin.com/jobs/view/2705235394/,https://media-exp1.licdn.com/dms/image/C4E0BAQ...,2022-03-16T00:39:45.465Z,https://www.linkedin.com/jobs/view/2705235394/,NaN
2,Software Engineer (Observability SaaS Platform...,"Singapore, Singapore",17.0,170.0,https://www.linkedin.com/in/ACoAACBJyQMBG7vI0Z...,2022-03-06T09:40:10.000Z,False,https://careers.tiktok.com/position/7059246670...,About TikTokTikTok is the leading destination ...,False,...,NaN,"Engineering, Information Technology",Internet,2022-04-05T09:40:09.000Z,Full-time,https://www.linkedin.com/jobs/view/2942975743/,https://media-exp1.licdn.com/dms/image/C510BAQ...,2022-03-16T00:39:48.373Z,https://www.linkedin.com/jobs/view/2942975743/,NaN
3,Data Engineer (EduHub),"Singapore, Singapore",16.0,126.0,NaN,2022-03-07T01:49:27.000Z,False,https://sggovterp.wd102.myworkdayjobs.com/en-U...,The Government Technology Agency (GovTech) aim...,False,...,NaN,Information Technology,Information Technology & Services,2022-04-06T01:49:25.000Z,Full-time,https://www.linkedin.com/jobs/view/2944331585/,https://media-exp1.licdn.com/dms/image/C4D0BAQ...,2022-03-16T00:39:51.212Z,https://www.linkedin.com/jobs/view/2944331585/,NaN
4,Hybrid Cloud Consultant,Singapore,19.0,113.0,NaN,2022-03-02T10:23:33.000Z,False,https://www.linkedin.com/job-apply/2935644471,"This is a client facing / presales role, worki...",True,...,NaN,"Information Technology, Consulting, Project Ma...",Information Technology & Services,2022-04-01T10:23:32.000Z,Full-time,https://www.linkedin.com/jobs/view/2935644471/,https://media-exp1.licdn.com/dms/image/C4D0BAQ...,2022-03-16T00:39:54.464Z,https://www.linkedin.com/jobs/view/2935644471/,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
802,Cloud Partner Success Manager (APAC),"Singapore, Singapore",7.0,51.0,NaN,2022-03-08T16:10:05.000Z,False,https://boards.greenhouse.io/acronis/jobs/5865...,Acronis is dedicated not just to cyber protect...,False,...,NaN,"Business Development, Sales",Computer Software,2022-04-07T16:10:04.000Z,Full-time,https://www.linkedin.com/jobs/view/2955575627/,https://media-exp1.licdn.com/dms/image/C510BAQ...,2022-03-15T04:33:58.399Z,https://www.linkedin.com/jobs/view/2955575627/,NaN
803,Java (Cloud Native) Developer,"Singapore, Singapore",4.0,32.0,https://www.linkedin.com/in/ACoAACtSD98BKfiq3v...,2022-02-09T07:14:42.000Z,False,https://www.linkedin.com/job-apply/2913886257,Project Description\nWe are looking for a tale...,False,...,NaN,"Engineering, Information Technology","Information Technology & Services, Computer So...",2022-03-11T07:14:41.000Z,Full-time,https://www.linkedin.com/jobs/view/2913886257/,https://media-exp1.licdn.com/dms/image/C560BAQ...,2022-03-15T04:34:01.335Z,https://www.linkedin.com/jobs/view/2913886257/,NaN
804,Cloud Native Developer (Azure),"Singapore, Singapore",3.0,17.0,https://www.linkedin.com/in/ACoAABNr

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 807 entries, 0 to 806
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   jobTitle             801 non-null    object 
 1   jobLocation          801 non-null    object 
 2   applicantsCount      773 non-null    float64
 3   viewsCount           801 non-null    float64
 4   jobPosterProfileUrl  164 non-null    object 
 5   postedAt             801 non-null    object 
 6   appliesClosed        801 non-null    object 
 7   applyUrl             768 non-null    object 
 8   jobDescription       801 non-null    object 
 9   remoteAllowed        801 non-null    object 
 10  companyName          801 non-null    object 
 11  companyUrl           765 non-null    object 
 12  matchedSkills        0 non-null      float64
 13  jobFunctions         783 non-null    object 
 14  jobIndustries        740 non-null    object 
 15  appliesClosedAt      801 non-null    obj

# Drop NA
Happens when: 
1. As Monster may direct users to the job posting in the company's website, the website format is different from Monster. 
Hence, data could not be scrapped for those job posting.
2. Usually if the salary is "Not specified" and years_experience is "undefined-undefined Years", the other fields will be NaN

In [26]:
# Drop job postings with errors (keep only postings with an empty error column)
df = df[df.error.isnull()]

# Date Posted

In [27]:
df['postedAt']

0      2022-03-11T13:49:39.000Z
1      2022-02-23T11:38:28.000Z
2      2022-03-06T09:40:10.000Z
3      2022-03-07T01:49:27.000Z
4      2022-03-02T10:23:33.000Z
                 ...           
802    2022-03-08T16:10:05.000Z
803    2022-02-09T07:14:42.000Z
804    2022-02-18T03:47:10.000Z
805    2022-03-10T11:29:42.000Z
806    2022-02-11T07:46:20.000Z
Name: postedAt, Length: 801, dtype: object

In [28]:
type(df['postedAt'][0])

str

In [29]:
# keep only the date
df['postedAt'] = [x[:10] for x in df['postedAt']]
df['postedAt']

0      2022-03-11
1      2022-02-23
2      2022-03-06
3      2022-03-07
4      2022-03-02
          ...    
802    2022-03-08
803    2022-02-09
804    2022-02-18
805    2022-03-10
806    2022-02-11
Name: postedAt, Length: 801, dtype: object

## Remove duplicates
Companies may post the same job a few times over a period of time. We will take the first post

In [31]:
# sort according to date
df.sort_values(by="postedAt", inplace=True)

In [34]:
# drop duplicates 
df.drop_duplicates(subset=['jobTitle', 'companyName', 'jobDescription'], keep='first', inplace=True)
df.reset_index(inplace=True, drop=True)
df

,jobTitle,jobLocation,applicantsCount,viewsCount,jobPosterProfileUrl,postedAt,appliesClosed,applyUrl,jobDescription,remoteAllowed,...,matchedSkills,jobFunctions,jobIndustries,appliesClosedAt,jobType,jobUrl,jobImageUrl,timestamp,query,error
0,Cloud Consumption Analyst,"Singapore, Singapore",19.0,368.0,NaN,2021-10-07,False,https://www.applytracking.com/x.aspx?method=di...,Job Description\n\nThe Cloud Consumption Analy...,False,...,NaN,Information Technology,Semiconductors,2022-03-20T04:06:49.000Z,Full-time,https://www.linkedin.com/jobs/view/2773797384/,https://media-exp1.licdn.com/dms/image/C560BAQ...,2022-03-14T04:07:59.226Z,https://www.linkedin.com/jobs/view/2773797384/,NaN
1,Senior Cloud Software Engineer,"Singapore, Singapore",45.0,580.0,https://www.linkedin.com/in/ACoAAADUtrsBMdWWpz...,2021-11-16,False,https://boards.greenhouse.io/motional/jobs/464...,We are seeking talented Senior Cloud Software ...,False,...,NaN,Engineering,Automotive,2022-07-30T07:51:08.000Z,Full-time,https://www.linkedin.com/jobs/view/2707798083/,https://media-exp1.licdn.com/dms/image/C560BAQ...,2022-03-14T11:19:42.877Z,https://www.linkedin.com/jobs/view/2707798083/,NaN
2,Pooling: Schneider Graduate Program Associate ...,"Singapore, Singapore",135.0,1087.0,NaN,2021-11-20,False,https://careers.se.com/jobs/0079VP?lang=en-us&...,Great People make Schneider Electric a great c...,False,...,NaN,Administrative,NaN,2022-04-06T05:41:12.000Z,Full-time,https://www.linkedin.com/jobs/view/2837955500/,https://media-exp1.licdn.com/dms/image/C4E0BAQ...,2022-03-14T04:07:32.792Z,https://www.linkedin.com/jobs/view/2837955500/,NaN
3,Infrastructure Analyst,"Singapore, Singapore",3.0,69.0,NaN,2021-11-20,False,https://www.capgemini.com/jobs/infrastructure-...,Who are we?\n\nCloud and Edge is a projects an...,False,...,NaN,Information Technology,Computer & Network Security,2022-04-06T03:54:32.000Z,Full-time,https://www.linkedin.com/jobs/view/2841040035/,https://media-exp1.licdn.com/dms/image/C4D0BAQ...,2022-03-15T20:06:55.957Z,https://www.linkedin.com/jobs/view/2841040035/,NaN
4,Junior Software Solutions Engineer,"Singapore, Singapore",10.0,102.0,NaN,2021-12-07,False,https://jobs.partnersgroup.com/job/Singapore-J...,What It's About\n\nT he Business Applications ...,False,...,NaN,"Engineering, Information Technology","Investment Management, Financial Services",2022-03-28T16:12:33.000Z,Full-time,https://www.linkedin.com/jobs/view/2821971962/,https://media-exp1.licdn.com/dms/image/C4D0BAQ...,2022-03-15T20:09:02.310Z,https://www.linkedin.com/jobs/view/2821971962/,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
783,"Inside Systems Engineer, Public Cloud","Singapore, Singapore",41.0,475.0,NaN,2022-03-15,False,https://veeam.wd3.myworkdayjobs.com/Veeam/job/...,The Inside Systems Engineer is responsible for...,False,...,NaN,Sales,Information Technology & Services,2022-06-12T03:27:09.000Z,Full-time,https://www.linkedin.com/jobs/view/2753103764/,https://media-exp1.licdn.com/dms/image/C560BAQ...,2022-03-16T00:44:49.434Z,https://www.linkedin.com/jobs/view/2753103764/,NaN
784,"Cloud Business Development Leader, APAC","Singapore, Singapore",49.0,433.0,NaN,2022-03-15,False,https://boards.greenhouse.io/rubrik/jobs/37440...,"Overview\n\nRubrik, the Zero Trust Data Securi...",False,...,NaN,"Business Development, Sales",Computer Software,2022-05-09T16:59:42.000Z,Full-time,https://www.linkedin.com/jobs/view/2869881810/,https://media-exp1.licdn.com/dms/image/C4D0BAQ...,2022-03-17T02:35:34.505Z,https://www.linkedin.com/jobs/view/2869881810/,NaN
785,Cloud Data Engineer,"Singapore, Singapore",90.0,396.0,https://www.linkedin.com/in/ACoAABVRW9oBB0fQxh...,2022-03-15,False,https://www.linkedin.com/job-apply/2886038785,NCS is a leading technology services firm with...,False,...,NaN,"Sales, Business Development, Information Techn...",Information Technology & Services,2022-04-21T03:11:17.000Z,Full-time,https://www.linkedin.com/jobs/view/2886038785/,https://media-ex

# Job Type


In [36]:
df['jobType'].unique()

array(['Full-time', 'Internship', 'Part-time', 'Contract', 'Temporary'],
      dtype=object)

In [37]:
# may need to change depending on the job types available in the platform
def clean_job_type(text):
    if 'full-time' in text.lower():
        return 'permanent'
    # take temporary and part-time as contract work
    elif ('contract' in text.lower()) or ('temporary' in text.lower()) or ('part-time' in text.lower()):
        return 'contract'
    elif 'internship' in text.lower():
        return 'intern'
    else: 
        return np.nan

In [38]:
df['jobType'] = df['jobType'].apply(lambda x: clean_job_type(str(x)))
df

,jobTitle,jobLocation,applicantsCount,viewsCount,jobPosterProfileUrl,postedAt,appliesClosed,applyUrl,jobDescription,remoteAllowed,...,matchedSkills,jobFunctions,jobIndustries,appliesClosedAt,jobType,jobUrl,jobImageUrl,timestamp,query,error
0,Cloud Consumption Analyst,"Singapore, Singapore",19.0,368.0,NaN,2021-10-07,False,https://www.applytracking.com/x.aspx?method=di...,Job Description\n\nThe Cloud Consumption Analy...,False,...,NaN,Information Technology,Semiconductors,2022-03-20T04:06:49.000Z,permanent,https://www.linkedin.com/jobs/view/2773797384/,https://media-exp1.licdn.com/dms/image/C560BAQ...,2022-03-14T04:07:59.226Z,https://www.linkedin.com/jobs/view/2773797384/,NaN
1,Senior Cloud Software Engineer,"Singapore, Singapore",45.0,580.0,https://www.linkedin.com/in/ACoAAADUtrsBMdWWpz...,2021-11-16,False,https://boards.greenhouse.io/motional/jobs/464...,We are seeking talented Senior Cloud Software ...,False,...,NaN,Engineering,Automotive,2022-07-30T07:51:08.000Z,permanent,https://www.linkedin.com/jobs/view/2707798083/,https://media-exp1.licdn.com/dms/image/C560BAQ...,2022-03-14T11:19:42.877Z,https://www.linkedin.com/jobs/view/2707798083/,NaN
2,Pooling: Schneider Graduate Program Associate ...,"Singapore, Singapore",135.0,1087.0,NaN,2021-11-20,False,https://careers.se.com/jobs/0079VP?lang=en-us&...,Great People make Schneider Electric a great c...,False,...,NaN,Administrative,NaN,2022-04-06T05:41:12.000Z,permanent,https://www.linkedin.com/jobs/view/2837955500/,https://media-exp1.licdn.com/dms/image/C4E0BAQ...,2022-03-14T04:07:32.792Z,https://www.linkedin.com/jobs/view/2837955500/,NaN
3,Infrastructure Analyst,"Singapore, Singapore",3.0,69.0,NaN,2021-11-20,False,https://www.capgemini.com/jobs/infrastructure-...,Who are we?\n\nCloud and Edge is a projects an...,False,...,NaN,Information Technology,Computer & Network Security,2022-04-06T03:54:32.000Z,permanent,https://www.linkedin.com/jobs/view/2841040035/,https://media-exp1.licdn.com/dms/image/C4D0BAQ...,2022-03-15T20:06:55.957Z,https://www.linkedin.com/jobs/view/2841040035/,NaN
4,Junior Software Solutions Engineer,"Singapore, Singapore",10.0,102.0,NaN,2021-12-07,False,https://jobs.partnersgroup.com/job/Singapore-J...,What It's About\n\nT he Business Applications ...,False,...,NaN,"Engineering, Information Technology","Investment Management, Financial Services",2022-03-28T16:12:33.000Z,permanent,https://www.linkedin.com/jobs/view/2821971962/,https://media-exp1.licdn.com/dms/image/C4D0BAQ...,2022-03-15T20:09:02.310Z,https://www.linkedin.com/jobs/view/2821971962/,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
783,"Inside Systems Engineer, Public Cloud","Singapore, Singapore",41.0,475.0,NaN,2022-03-15,False,https://veeam.wd3.myworkdayjobs.com/Veeam/job/...,The Inside Systems Engineer is responsible for...,False,...,NaN,Sales,Information Technology & Services,2022-06-12T03:27:09.000Z,permanent,https://www.linkedin.com/jobs/view/2753103764/,https://media-exp1.licdn.com/dms/image/C560BAQ...,2022-03-16T00:44:49.434Z,https://www.linkedin.com/jobs/view/2753103764/,NaN
784,"Cloud Business Development Leader, APAC","Singapore, Singapore",49.0,433.0,NaN,2022-03-15,False,https://boards.greenhouse.io/rubrik/jobs/37440...,"Overview\n\nRubrik, the Zero Trust Data Securi...",False,...,NaN,"Business Development, Sales",Computer Software,2022-05-09T16:59:42.000Z,permanent,https://www.linkedin.com/jobs/view/2869881810/,https://media-exp1.licdn.com/dms/image/C4D0BAQ...,2022-03-17T02:35:34.505Z,https://www.linkedin.com/jobs/view/2869881810/,NaN
785,Cloud Data Engineer,"Singapore, Singapore",90.0,396.0,https://www.linkedin.com/in/ACoAABVRW9oBB0fQxh...,2022-03-15,False,https://www.linkedin.com/job-apply/2886038785,NCS is a leading technology services firm with...,False,...,NaN,"Sales, Business Development, Information Techn...",Information Technology & Services,2022-04-21T03:11:17.000Z,permanent,https://www.linkedin.com/jobs/view/2886038785/,https://media-ex

# Remove unnecessary columns

In [58]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 788 entries, 0 to 787
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   jobTitle             788 non-null    object 
 1   jobLocation          788 non-null    object 
 2   applicantsCount      760 non-null    float64
 3   viewsCount           788 non-null    float64
 4   jobPosterProfileUrl  161 non-null    object 
 5   postedAt             788 non-null    object 
 6   appliesClosed        788 non-null    object 
 7   applyUrl             755 non-null    object 
 8   jobDescription       788 non-null    object 
 9   remoteAllowed        788 non-null    object 
 10  companyName          788 non-null    object 
 11  companyUrl           753 non-null    object 
 12  matchedSkills        0 non-null      float64
 13  jobFunctions         770 non-null    object 
 14  jobIndustries        728 non-null    object 
 15  appliesClosedAt      788 non-null    obj

In [62]:
# all these columns are irrelevant, location is all singapore
df.drop(columns = ['error', 'matchedSkills', 'appliesClosedAt', 'viewsCount', 'jobLocation', 'jobPosterProfileUrl', 'appliesClosed', 'applyUrl', 'appliesClosedAt', 'jobImageUrl', 'query', 'error', 'timestamp'], inplace=True)

# Rename columns

In [63]:
df.columns

Index(['jobTitle', 'applicantsCount', 'postedAt', 'jobDescription',
       'remoteAllowed', 'companyName', 'companyUrl', 'jobFunctions',
       'jobIndustries', 'jobType', 'jobUrl'],
      dtype='object')

In [64]:
rename = {
    'jobTitle': 'job_title',
    'jobUrl': 'url',
    'postedAt': 'date_posted',
    'companyName': 'company_name',
    'jobIndustries': 'industry',
    'jobFunctions': 'job_functions',
    'jobDescription': 'job_desc',
    'jobType': 'job_type'
}

df.rename(columns=rename, inplace=True)

In [66]:
# Change directory and filename
save_dir = "/Users/joshuawong/Documents/GitHub/job-posting-analysis/data/cleaned"
df.to_csv(f'{save_dir}/LinkedIn.csv', index=False)